In [217]:
import pandas as pd
from pymongo import MongoClient
import folium
from pandas.io.json import json_normalize
from folium.plugins import HeatMap
from folium.plugins import HeatMap

In [218]:
client = MongoClient()
db = client.Companies
comp = db.Companies


In [219]:
df = pd.DataFrame(comp.find({'category_code':'web','offices.0.country_code':'USA'},
                                {'_id':0, 'name':1, 'offices':1, 'category_code':1, 'founded_year':1}))
df.offices[0]

[{'description': '',
  'address1': '710 - 2nd Avenue',
  'address2': 'Suite 1100',
  'zip_code': '98104',
  'city': 'Seattle',
  'state_code': 'WA',
  'country_code': 'USA',
  'latitude': 47.603122,
  'longitude': -122.333253},
 {'description': '',
  'address1': '270 Lafayette Street',
  'address2': 'Suite 505',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.7237306,
  'longitude': -73.9964312}]

In [220]:
df['offices'] = df['offices'].apply(lambda x: x[0])
df.head()

,name,category_code,founded_year,offices
0,Wetpaint,web,2005.0,"{'description': '', 'address1': '710 - 2nd Ave..."
1,Postini,web,1999.0,"{'description': None, 'address1': '959 Skyway ..."
2,Geni,web,2006.0,"{'description': 'Headquarters', 'address1': '9..."
3,Fox Interactive Media,web,1979.0,"{'description': '', 'address1': '407 N Maple D..."
4,StumbleUpon,web,2002.0,"{'description': '', 'address1': '', 'address2'..."


In [221]:
df = pd.concat([json_normalize(df['offices']), df[['name' ,'category_code']]], axis=1)
df = df.dropna().reset_index()
df = df[['name','city','state_code','latitude','longitude','category_code','country_code']]
df

,name,city,state_code,latitude,longitude,category_code,country_code
0,Wetpaint,Seattle,WA,47.603122,-122.333253,web,USA
1,Geni,West Hollywood,CA,34.090368,-118.393064,web,USA
2,Fox Interactive Media,Beverly Hills,CA,34.076179,-118.394170,web,USA
3,StumbleUpon,San Francisco,CA,37.775196,-122.419204,web,USA
4,eBay,San Jose,CA,37.295005,-121.930035,web,USA
...,...,...,...,...,...,...,...
962,Wolfram Alpha,Champaign,IL,40.097071,-88.245638,web,USA
963,CalFinder,Oakland,CA,37.797862,-122.277506,web,USA
964,VestMatch,Oakland,CA,37.810448,-122.239864,web,USA
965,VitalLock,Cambridge,MA,42.362856,-71.083762,web,USA


In [222]:
df

,name,city,state_code,latitude,longitude,category_code,country_code
0,Wetpaint,Seattle,WA,47.603122,-122.333253,web,USA
1,Geni,West Hollywood,CA,34.090368,-118.393064,web,USA
2,Fox Interactive Media,Beverly Hills,CA,34.076179,-118.394170,web,USA
3,StumbleUpon,San Francisco,CA,37.775196,-122.419204,web,USA
4,eBay,San Jose,CA,37.295005,-121.930035,web,USA
...,...,...,...,...,...,...,...
962,Wolfram Alpha,Champaign,IL,40.097071,-88.245638,web,USA
963,CalFinder,Oakland,CA,37.797862,-122.277506,web,USA
964,VestMatch,Oakland,CA,37.810448,-122.239864,web,USA
965,VitalLock,Cambridge,MA,42.362856,-71.083762,web,USA


In [223]:
data = list(zip(df['latitude'], df['longitude']))
data = [i for i in data]

"""
Cliente que quiere concentrarse en donde se encuentran la mayoria de las empresas web dentro de los EUA, por lo que lo primero que hicimos es sacar el mapa de calor para encontrar los estados con mayor cantidad de empresas con categoria "web"

"""

In [224]:
mapa=folium.Map(location=[37.295005, -121.930035 ])
mapa.add_child(HeatMap(data))

In [225]:
# valores que mas se repiten en el dataframe por estado, por lo que vemos que aunque en el mapa parece que existen
# mas empresas en la costa este, la mayor parte de las empresas de desarrollo web se encuentran en california(CA)
new_df = pd.DataFrame(df['state_code'].value_counts())       
new_df

,state_code
CA,433
NY,119
WA,49
MA,41
TX,34
FL,33
IL,31
GA,22
VA,19
PA,17


# suponemos que el cliente elige ny ya que es un punto medio entre tener muchas empresas que lleguen a competir y poder tener una red de contactos del mismo ambito de negocios

In [231]:
city_df = df.loc[df['state_code'] == 'NY'].reset_index()
city_df

,level_0,index,name,city,state_code,latitude,longitude,category_code,country_code
0,0,5,Viacom,New York,NY,40.757725,-73.986011,web,USA
1,1,35,Forbes,New York,NY,40.736416,-73.993302,web,USA
2,2,39,AOL,New York,NY,40.731132,-73.991931,web,USA
3,3,59,Nielsen,New York,NY,40.731118,-73.991933,web,USA
4,4,74,Tutor,New York,NY,40.707921,-74.004792,web,USA
...,...,...,...,...,...,...,...,...,...
114,114,918,CollectiveSys,Flushing,NY,40.760922,-73.830489,web,USA
115,115,927,MyWealth,New York,NY,40.703826,-74.008328,web,USA
116,116,937,Tinychat,Glen Cove,NY,42.375641,-72.519691,web,USA
117,117,944,CogniFit,New York,NY,47.602337,-122.285011,web,USA


In [234]:
mapa.add_child(Circle(city_df,radius=15))

NameError: name 'Circle' is not defined